In [1]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin


!nvcc --version

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-cqp3f2kf
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-cqp3f2kf
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp36-none-any.whl size=4307 sha256=9afe78217815a424b45379c8f214252d5af65e813816c47013a89281f312bb15
  Stored in directory: /tmp/pip-ephem-wheel-cache-s6yf4cet/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin
created output directory at /content/src
Out bin /content/result.out
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243


**Q2 Sum of two arrays**


In [ ]:
%%cu
#include <stdio.h>
#include<random>
#define HANDLE_ERROR( err ) ( HandleError( err, __FILE__, __LINE__ ) )

static void HandleError( cudaError_t err, const char *file, int line )
{
    if (err != cudaSuccess)
      {
        printf( "%s in %s at line %d\n", cudaGetErrorString( err ),
                file, line );
        exit( EXIT_FAILURE );
    }
}

const short N = 10 ;

__global__ void Vector_Addition ( const int *dev_a , const int *dev_b , int *dev_c)
{
      //Get the id of thread within a block
      unsigned short tid = threadIdx.x ;
     
     // check the boundry condition for the threads
      if ( tid < N ) 
            dev_c [tid] = dev_a[tid] + dev_b[tid] ;
//printf("%p \n", &N);

}


int main (void)
{

    srand(0);
    int Host_a[N], Host_b[N], Host_c[N];

    int *dev_a , *dev_b, *dev_c ;

    HANDLE_ERROR ( cudaMalloc((void **)&dev_a , N*sizeof(int) ) );
    HANDLE_ERROR ( cudaMalloc((void **)&dev_b , N*sizeof(int) ) );
    HANDLE_ERROR ( cudaMalloc((void **)&dev_c , N*sizeof(int) ) );
    
    
    for ( int i = 0; i <N ; i++ )
    {
        Host_a[i] = rand()%100 ;
        Host_b[i] = rand()%100 ; 
    }

    HANDLE_ERROR (cudaMemcpy (dev_a , Host_a , N*sizeof(int) , cudaMemcpyHostToDevice));
    HANDLE_ERROR (cudaMemcpy (dev_b , Host_b , N*sizeof(int) , cudaMemcpyHostToDevice));

    Vector_Addition <<< 1, N  >>> (dev_a , dev_b , dev_c ) ;

    HANDLE_ERROR (cudaMemcpy(Host_c , dev_c , N*sizeof(int) , cudaMemcpyDeviceToHost));

    for ( int i = 0; i<N; i++ )
    printf ("%d + %d = %d\n", Host_a[i] , Host_b[i] , Host_c[i] ) ;

    cudaFree (dev_a) ;
    cudaFree (dev_b) ;
    cudaFree (dev_c) ;
    printf("%p \n", &N);

    return 0 ;

}


83 + 86 = 169
77 + 15 = 92
93 + 35 = 128
86 + 92 = 178
49 + 21 = 70
62 + 27 = 89
90 + 59 = 149
63 + 26 = 89
40 + 26 = 66
72 + 36 = 108
0x55ed71d9c0fe 



**Q1 Sum of Array of Elements**

In [ ]:
%%cu
#include <stdio.h>
#include<random>
#define HANDLE_ERROR( err ) ( HandleError( err, __FILE__, __LINE__ ) )

static void HandleError( cudaError_t err, const char *file, int line )
{
    if (err != cudaSuccess)
      {
        printf( "%s in %s at line %d\n", cudaGetErrorString( err ),
                file, line );
        exit( EXIT_FAILURE );
    }
}

const short N = 10 ;


__global__ void Vector_Addition ( const int *dev_a, int *dev_b)
{

    unsigned short tid = threadIdx.x ;
    
    int tmp=0;
    if(tid < N)
        dev_b[0] += dev_a[tid] ;
    printf("%d ",dev_b[0]);
          
}


int main (void)
{

    srand(0);
    int Host_a[N], Host_b[2];

    int *dev_a, *dev_b;
    

    HANDLE_ERROR ( cudaMalloc((void **)&dev_a , N*sizeof(int) ) );
    HANDLE_ERROR ( cudaMalloc((void **)&dev_b , 2*sizeof(int) ) );
    
    int sum=0;
    for ( int i = 0; i <N ; i++ )
    {
        Host_a[i] = rand()%100 ;
        printf("%d ",Host_a[i]);
        sum += Host_a[i];
    }
    printf("\nSum in cpu is: %d\n",sum);
    HANDLE_ERROR (cudaMemcpy (dev_a , Host_a , N*sizeof(int) , cudaMemcpyHostToDevice));
  
    //Vector_Addition <<< 1, N  >>> (dev_a) ;
     Vector_Addition <<< 1, N  >>> (dev_a , dev_b) ;
     cudaDeviceSynchronize();
   HANDLE_ERROR (cudaMemcpy(Host_b , dev_b , 2*sizeof(int) , cudaMemcpyDeviceToHost));

    
    printf ("hello%d \n",  Host_b[0] ) ;

    cudaFree (dev_a) ;
    
    cudaFree (dev_b) ;
    printf("%p \n", &N);

    return 0 ;

}


83 86 77 15 93 35 86 92 49 21 
Sum in cpu is: 637
83 86 77 15 93 35 86 92 49 21 hello83 
0x55c0feb5fffe 



**Q3 Matrix Multiplication**

In [ ]:
%%cu
#include <stdio.h>
#include<random>
#define HANDLE_ERROR( err ) ( HandleError( err, __FILE__, __LINE__ ) )

static void HandleError( cudaError_t err, const char *file, int line )
{
    if (err != cudaSuccess)
      {
        printf( "%s in %s at line %d\n", cudaGetErrorString( err ),
                file, line );
        exit( EXIT_FAILURE );
    }
}

const short M = 10, N = 5, P = 3 ;

__global__ void Vector_Addition ( const int *dev_a , const int *dev_b , int *dev_c)
{
      
    
    int col = blockIdx.x*blockDim.x+threadIdx.x;
    int row = col/P;
    col = col%P;
    //printf("Row: %d, col:%d \n", row, col);

    int tmp=0;
    if(row< M && col < P)
    {
        for(int i=0; i<N; i++)
        {
            tmp += dev_a[row * N + i] * dev_b[i * P + col];
        }
     
    }
    dev_c[row* P + col] = tmp;
    
}


int main (void)
{

    srand(0);
    int Host_a[M][N], Host_b[N][P], Host_c[M][P];
    int c[M][P];
    int *dev_a , *dev_b, *dev_c ;

    HANDLE_ERROR ( cudaMalloc((void **)&dev_a , M*N*sizeof(int) ) );
    HANDLE_ERROR ( cudaMalloc((void **)&dev_b , N*P*sizeof(int) ) );
    HANDLE_ERROR ( cudaMalloc((void **)&dev_c , M*P*sizeof(int) ) );
    
    printf("A:\n");
    for ( int i = 0; i <M; i++ )
    {
       for(int j=0 ; j<N;j++)
       {
         Host_a[i][j] = rand()%20 ;
         printf("%d ",Host_a[i][j]);
       }
       printf("\n");
    }
    printf("\n");
 
    printf("B:\n");
    for ( int i = 0; i <N; i++ )
    {
        for(int j=0 ; j<P;j++)
        {
            Host_b[i][j] = rand()%20; 
         printf("%d ",Host_b[i][j]);
        }
     printf("\n");
    }
    printf("\nMultiplication in cpu:\n");
    for(int i=0;i<M;i++)
    {
        for(int j=0;j<P;j++)
        {
            int tmp=0;
            for(int k=0;k<N;k++)
            {
                tmp +=Host_a[i][k]*Host_b[k][j];
            }
            c[i][j] = tmp;
            printf("%d ",c[i][j]);
        }
     printf("\n");
    }
    printf("\n");
    HANDLE_ERROR (cudaMemcpy (dev_a , Host_a , M*N*sizeof(int) , cudaMemcpyHostToDevice));
    HANDLE_ERROR (cudaMemcpy (dev_b , Host_b , N*P*sizeof(int) , cudaMemcpyHostToDevice));

    Vector_Addition <<< 1, M*P>>> (dev_a , dev_b , dev_c ) ;
    cudaDeviceSynchronize();
    HANDLE_ERROR (cudaMemcpy(Host_c , dev_c , M* P*sizeof(int) , cudaMemcpyDeviceToHost));
    printf("\nMultiplication in Gpu:\n");
    for ( int i = 0; i<M; i++ )
    {
        for(int j=0;j<P;j++)
            printf ("%d ",  Host_c[i][j] ) ;
     printf("\n");
    }
    cudaFree (dev_a) ;
    cudaFree (dev_b) ;
    cudaFree (dev_c) ;

    return 0 ;

}


A:
3 6 17 15 13 
15 6 12 9 1 
2 7 10 19 3 
6 0 6 12 16 
11 8 7 9 2 
10 2 3 7 15 
9 2 2 18 9 
7 13 16 11 2 
9 13 1 19 4 
17 18 4 15 10 

B:
13 6 11 
0 16 13 
2 10 16 
1 5 5 
4 7 16 

Multiplication in cpu:
140 450 666 
232 358 496 
77 340 416 
166 268 478 
174 323 414 
203 262 459 
175 259 391 
142 479 589 
154 395 443 
284 575 720 


Multiplication in Gpu:
140 450 666 
232 358 496 
77 340 416 
166 268 478 
174 323 414 
203 262 459 
175 259 391 
142 479 589 
154 395 443 
284 575 720 



**Q5 BFS in GPU**


In [ ]:
%%cu
#include <stdio.h>
#include<random>
#include<bits/stdc++.h>

using namespace std;
#define HANDLE_ERROR( err ) ( HandleError( err, __FILE__, __LINE__ ) )

static void HandleError( cudaError_t err, const char *file, int line )
{
    if (err != cudaSuccess)
      {
        printf( "%s in %s at line %d\n", cudaGetErrorString( err ),
                file, line );
        exit( EXIT_FAILURE );
    }
}

const int N = 10;   // num vertexes


__global__ void bfs_kernel(const int *d_mat, int* d_c, int *d_vis, int*d_fa)
{
    int tid = threadIdx.x ;
    if (d_fa[tid])
    {
        d_fa[tid] = 0;
        d_vis[tid] = 1;
        for(int i=0; i<N; i++)
        {
           if( d_mat[tid*N + i] && !d_vis[i])
           {
                d_c[i] = d_c[tid]+1;
                d_fa[i] = 1;
           }
        }
    }
}


int main (void)
{

    srand(0);
    int mat[N][N];  // host adjacency matrix
   
///////////////////////////////////////////////////////
// random matrix

    printf("Adjacency matrix:\n");
    for ( int i = 0; i <N ; i++ )
    {
        mat[i][i] = 0;
        for(int j=0;j<N;j++)
        {
            if(j>i)
            {
            mat[i][j] = rand()%2;
            mat[j][i] = mat[i][j];
            }
            printf("%d ",mat[i][j]);
        }
        printf("\n");
    }
    int s = 0;   // source vertex
 
 /////////////////////////////////////////////////////////
 // bfs in cpu 
    
    queue<int> q;
    int vis[N] = {0};
    int c[N];
    for(int i=0;i<N;i++)
    {
       c[i] = INT_MAX ;
    }
    
    c[s] = 0;
    q.push(s);
    
    while(!q.empty())
    {
        int x = q.front();
        q.pop();
        vis[x] = 1;
        for(int i=0; i<N; i++)
        {
            if(mat[x][i] && !vis[i])
            {
                q.push(i);
                c[i] = min (c[i] , c[x] + 1);
            }
        }
    }
    
    printf("\nCost in CPU: ");
    for(int i=0; i<N; i++)
    {
        printf("%d ",c[i]);
    }
/////////////////////////////////////////////////////////
// bfs in gpu
    
    int *d_mat;  //device adj matrix
    int fa[N];
    int *d_fa, *d_vis, *d_c; 
 
    HANDLE_ERROR ( cudaMalloc((void **)&d_mat , N*N*sizeof(int) ) );
    HANDLE_ERROR ( cudaMalloc((void **)&d_vis , N*sizeof(int) ) );
    HANDLE_ERROR ( cudaMalloc((void **)&d_c , N*sizeof(int) ) );
    HANDLE_ERROR ( cudaMalloc((void **)&d_fa , N*sizeof(int) ) );
    for(int i=0; i<N; i++)
    {
        vis[i] = 0;
        fa[i] = 0;
        c[i] = INT_MAX;
    }

    fa[s] = 1;
    c[s] = 0;
 
    HANDLE_ERROR ( cudaMemcpy (d_mat , mat , N*N*sizeof(int) , cudaMemcpyHostToDevice));
    HANDLE_ERROR ( cudaMemcpy (d_c , c , N*sizeof(int) , cudaMemcpyHostToDevice));
    HANDLE_ERROR ( cudaMemcpy (d_vis , vis , N*sizeof(int) , cudaMemcpyHostToDevice));
    HANDLE_ERROR ( cudaMemcpy (d_fa , fa , N*sizeof(int) , cudaMemcpyHostToDevice));
    
    for(int j=0;j<N;j++)
    {
        while(fa[j])
        {
            for(int i=0;i<N;i++)
            {
                bfs_kernel<<<1,N>>> (d_mat, d_c, d_vis, d_fa);
                HANDLE_ERROR ( cudaMemcpy (fa , d_fa , N*sizeof(int) ,  cudaMemcpyDeviceToHost));
            }
        }
    }
    HANDLE_ERROR ( cudaMemcpy (vis , d_vis , N*sizeof(int) ,  cudaMemcpyDeviceToHost));
    HANDLE_ERROR ( cudaMemcpy (c , d_c , N*sizeof(int) ,  cudaMemcpyDeviceToHost));

    printf("\nCost in GPU: ");
    for(int i=0; i<N; i++)
    {
        printf("%d ",c[i]);
    }
    cudaFree (d_mat) ;
    cudaFree (d_vis) ;
    cudaFree (d_c) ;
    cudaFree (d_fa) ;
    return 0 ;

}


Adjacency matrix:
0 1 0 1 1 1 1 0 0 1 
1 0 1 0 1 0 1 1 0 0 
0 1 0 0 0 0 1 0 1 1 
1 0 0 0 0 0 0 1 1 1 
1 1 0 0 0 1 0 0 0 1 
1 0 0 0 1 0 1 1 0 1 
1 1 1 0 0 1 0 0 1 1 
0 1 0 1 0 1 0 0 1 1 
0 0 1 1 0 0 1 1 0 0 
1 0 1 1 1 1 1 1 0 0 

Cost in CPU: 0 1 2 1 1 1 1 2 2 1 
Cost in GPU: 0 1 2 1 1 1 1 2 2 1 


**Q6 Single Source Shortest Path**

In [ ]:
%%cu
#include <stdio.h>
#include<random>
#include<bits/stdc++.h>

using namespace std;
#define HANDLE_ERROR( err ) ( HandleError( err, __FILE__, __LINE__ ) )

static void HandleError( cudaError_t err, const char *file, int line )
{
    if (err != cudaSuccess)
      {
        printf( "%s in %s at line %d\n", cudaGetErrorString( err ),
                file, line );
        exit( EXIT_FAILURE );
    }
}

const int N = 10;   // num vertexes


int minDistance(int dist[], bool sptSet[]) 
{ 
    int min = INT_MAX, min_index; 
  
    for (int v = 0; v < N; v++) 
        if (sptSet[v] == false && dist[v] <= min) 
            min = dist[v], min_index = v; 
  
    return min_index; 
} 



__global__ void sssp_kernel1(const int *d_mat, int* d_c, int *d_m, int*d_u)
{
    int tid = threadIdx.x ;
    if (d_m[tid])
    {
        d_m[tid] = 0;
        for(int i=0; i<N; i++)
        {
           if( d_mat[tid*N + i]>0 &&  d_u[i] > d_c[tid]+d_mat[tid*N + i])
           {
                d_u[i] = d_c[tid] + d_mat[tid*N + i];
           }
        }
    }
    //printf("hi");
 
}

__global__ void sssp_kernel2( int* d_c, int *d_m, int*d_u)
{ 
    int tid = threadIdx.x ;
 //printf("Hello");
    if (d_c[tid] > d_u[tid])
    {
       d_c[tid] = d_u[tid];
       d_m[tid] = 1;
    }
    d_u[tid] = d_c[tid];
}

int main (void)
{

    srand(0);
    int mat[N][N];  // host adjacency matrix
   
///////////////////////////////////////////////////////
// random matrix

    printf("Adjacency matrix:\n");
    for ( int i = 0; i <N ; i++ )
    {
        mat[i][i] = 0;
        for(int j=0;j<N;j++)
        {
            if(j>i)
            {
                mat[i][j] = rand()%10;
                mat[j][i] = mat[i][j];
            }
            printf("%d ",mat[i][j]);
        }
        printf("\n");
    }
    int s = 0;   // source vertex
 
 /////////////////////////////////////////////////////////
 // sssp in cpu dijkstra's algorithm
    
    int dist[N];
    bool sptSet[N]; 
 
    for (int i = 0; i < N; i++) 
        dist[i] = INT_MAX, sptSet[i] = false; 
  
    
    dist[s] = 0; 
  
    for (int count = 0; count < N - 1; count++) { 
        
        int u = minDistance(dist, sptSet); 
        sptSet[u] = true; 
  
 
        for (int v = 0; v < N; v++) 
            if (!sptSet[v] && mat[u][v] && dist[u] != INT_MAX 
                && dist[u] + mat[u][v] < dist[v]) 
                dist[v] = dist[u] + mat[u][v]; 
    } 
  
    printf("\nCost in CPU: ");
     for (int i = 0; i < N; i++) 
        printf("%d ", dist[i]); 
    printf("\n");
 
/////////////////////////////////////////////////////////
// bfs in gpu
    
    int *d_mat;  //device adj matrix
    int m[N], c[N], u[N];
    int *d_m, *d_c, *d_u; 
 
    HANDLE_ERROR ( cudaMalloc((void **)&d_mat , N*N*sizeof(int) ) );
    HANDLE_ERROR ( cudaMalloc((void **)&d_m , N*sizeof(int) ) );
    HANDLE_ERROR ( cudaMalloc((void **)&d_c , N*sizeof(int) ) );
    HANDLE_ERROR ( cudaMalloc((void **)&d_u , N*sizeof(int) ) );
    for(int i=0; i<N; i++)
    {
        m[i] = 0;
        u[i] = INT_MAX;
        c[i] = INT_MAX;
    }

    m[s] = 1;
    c[s] = 0;
    u[s] = 0;
 
    HANDLE_ERROR ( cudaMemcpy (d_mat , mat , N*N*sizeof(int) , cudaMemcpyHostToDevice));
    HANDLE_ERROR ( cudaMemcpy (d_c , c , N*sizeof(int) , cudaMemcpyHostToDevice));
    HANDLE_ERROR ( cudaMemcpy (d_m , m , N*sizeof(int) , cudaMemcpyHostToDevice));
    HANDLE_ERROR ( cudaMemcpy (d_u , u , N*sizeof(int) , cudaMemcpyHostToDevice));
    
    for(int i=0;i<N;i++)
    {
        while(m[i])
        {
            for(int j=0;j<N;j++)
            {
                sssp_kernel1<<<1,N>>> (d_mat, d_c, d_m, d_u);
                cudaDeviceSynchronize();
               
                HANDLE_ERROR ( cudaMemcpy (m , d_m , N*sizeof(int) ,  cudaMemcpyDeviceToHost));
                HANDLE_ERROR ( cudaMemcpy (u , d_u , N*sizeof(int) ,  cudaMemcpyDeviceToHost));
                HANDLE_ERROR ( cudaMemcpy (c , d_c , N*sizeof(int) ,  cudaMemcpyDeviceToHost));
                HANDLE_ERROR ( cudaMemcpy (d_c , c , N*sizeof(int) , cudaMemcpyHostToDevice));
                HANDLE_ERROR ( cudaMemcpy (d_m , m , N*sizeof(int) , cudaMemcpyHostToDevice));
                HANDLE_ERROR ( cudaMemcpy (d_u , u , N*sizeof(int) , cudaMemcpyHostToDevice));
                
                sssp_kernel2<<<1,N>>> (d_c, d_m, d_u);
                cudaDeviceSynchronize();
                HANDLE_ERROR ( cudaMemcpy (m , d_m , N*sizeof(int) ,  cudaMemcpyDeviceToHost));
                HANDLE_ERROR ( cudaMemcpy (u , d_u , N*sizeof(int) ,  cudaMemcpyDeviceToHost));
                HANDLE_ERROR ( cudaMemcpy (c , d_c , N*sizeof(int) ,  cudaMemcpyDeviceToHost));
                HANDLE_ERROR ( cudaMemcpy (d_c , c , N*sizeof(int) , cudaMemcpyHostToDevice));
                HANDLE_ERROR ( cudaMemcpy (d_m , m , N*sizeof(int) , cudaMemcpyHostToDevice));
                HANDLE_ERROR ( cudaMemcpy (d_u , u , N*sizeof(int) , cudaMemcpyHostToDevice));
             
            }
        }
    }
    HANDLE_ERROR ( cudaMemcpy (m , d_m , N*sizeof(int) ,  cudaMemcpyDeviceToHost));
    HANDLE_ERROR ( cudaMemcpy (u , d_u , N*sizeof(int) ,  cudaMemcpyDeviceToHost));
    HANDLE_ERROR ( cudaMemcpy (c , d_c , N*sizeof(int) ,  cudaMemcpyDeviceToHost));

    printf("\nCost in GPU: ");
    for(int i=0; i<N; i++)
    {
        printf("%d ",c[i]);
    }
    cudaFree (d_mat) ;
    cudaFree (d_u) ;
    cudaFree (d_c) ;
    cudaFree (d_m) ;
    return 0 ;

}

Adjacency matrix:
0 3 6 7 5 3 5 6 2 9 
3 0 1 2 7 0 9 3 6 0 
6 1 0 6 2 6 1 8 7 9 
7 2 6 0 2 0 2 3 7 5 
5 7 2 2 0 9 2 2 8 9 
3 0 6 0 9 0 7 3 6 1 
5 9 1 2 2 7 0 2 9 3 
6 3 8 3 2 3 2 0 1 9 
2 6 7 7 8 6 9 1 0 4 
9 0 9 5 9 1 3 9 4 0 

Cost in CPU: 0 3 4 5 5 3 5 3 2 4 

Cost in GPU: 0 3 4 5 5 3 5 3 2 4 


**Q7 Sort the array using thrust library**

In [11]:
%%cu
#include <thrust/host_vector.h>
#include <thrust/device_vector.h>
#include <thrust/generate.h>
#include <thrust/sort.h>
#include <thrust/copy.h>
#include <cstdlib>
int main(void)
{
 
    thrust::host_vector<int> h_vec(32 << 1);

    thrust::generate(h_vec.begin(), h_vec.end(), rand);
    printf("Unsorted array: ");
    for(int i=0;i<h_vec.size();i++)
    {
        printf("%d ",h_vec[i]);
    }
    printf("\n");
 
    // transfer data to the device and sort
    thrust::device_vector<int> d_vec = h_vec;
    thrust::sort(d_vec.begin(), d_vec.end());
    
    // transfer data back to host
    thrust::copy(d_vec.begin(), d_vec.end(), h_vec.begin());
    printf("Sorted array: ");
    for(int i=0;i<h_vec.size();i++)
    {
        printf("%d ",h_vec[i]);
    }
    printf("\n");
    return 0;
}

Unsorted array: 1804289383 846930886 1681692777 1714636915 1957747793 424238335 719885386 1649760492 596516649 1189641421 1025202362 1350490027 783368690 1102520059 2044897763 1967513926 1365180540 1540383426 304089172 1303455736 35005211 521595368 294702567 1726956429 336465782 861021530 278722862 233665123 2145174067 468703135 1101513929 1801979802 1315634022 635723058 1369133069 1125898167 1059961393 2089018456 628175011 1656478042 1131176229 1653377373 859484421 1914544919 608413784 756898537 1734575198 1973594324 149798315 2038664370 1129566413 184803526 412776091 1424268980 1911759956 749241873 137806862 42999170 982906996 135497281 511702305 2084420925 1937477084 1827336327 
Sorted array: 35005211 42999170 135497281 137806862 149798315 184803526 233665123 278722862 294702567 304089172 336465782 412776091 424238335 468703135 511702305 521595368 596516649 608413784 628175011 635723058 719885386 749241873 756898537 783368690 846930886 859484421 861021530 982906996 1025202362 105996